<a href="https://colab.research.google.com/github/moisesoliv/sent-detect/blob/master/3predict_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/tcc')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import tensorflow as tf
import pickle
import pandas as pd

from modules.preprocessing import *

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
class EmotionDetect:
    def __init__(self, vectors_name, w_folder):
        self.w_folder = w_folder
        self.size_word2vec = 300
        self.max_sequences = 35
        # self.vectors = pickle.load(open('w2v_stop'+str(self.size_word2vec)+'.pkl', 'rb'))
        self.vectors = pickle.load(open(vectors_name, "rb"))
        self.emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
        self.tokenizer = Tokenizer()
        self.loadModels()

    def model_gru(self, embedding_matrix, embed_size):
        # load json and create model
        json_file = open("models/bi-gru300.json", "r")
        loaded_model_json = json_file.read()
        json_file.close()
        return tf.keras.models.model_from_json(loaded_model_json)

    def loadModels(self):
        self.models = dict()
        for s in self.emotions:
            self.models[s] = self.model_gru(self.vectors.wv.vectors, self.size_word2vec)
            self.models[s].load_weights("models/" + s + "_weights.h5")

    def getIndex(self, t):
        try:
            return self.vectors.wv.vocab[t].index
        except:
            return 0

    def predict(self, text):
        x = self.tokenizer.cleanText(text)
        x = [self.getIndex(t) for t in x]
        x = tf.keras.preprocessing.sequence.pad_sequences(
            [x], maxlen=self.max_sequences
        )
        return dict(
            zip(self.emotions, [self.models[s].predict(x)[0][0] for s in self.emotions])
        )

In [ ]:
classifier = EmotionDetect("models/ft_300.pkl", "models")

In [ ]:
classifier.predict("hello world!")

{'anger': 1.758303e-17,
 'disgust': 1.2089988e-15,
 'fear': 8.738927e-19,
 'joy': 0.09627417,
 'sadness': 2.6074428e-23,
 'surprise': 7.228096e-06}

In [ ]:
def predict_file(file):
    try:
        tweets = pd.read_csv(file)
    except:
        tweets = pd.read_pickle(file)

    l = list()
    for index, t in tweets.iterrows():
        emo = classifier.predict(t.text)
        l.append(
            {
                "date": t["date"],
                "id": t["id"],
                "text": t["text"],
                "favorites": t["favorites"],
                "retweets": t["retweets"],
                "replies": t["replies"],

                "anger": int(emo["anger"] > 0.12),
                "disgust": int(emo["disgust"] > 0.44),
                "fear": int(emo["fear"] > 0.1),
                "joy": int(emo["joy"] > 0.1),
                "sadness": int(emo["sadness"] > 0.11),
                "surprise": int(emo["surprise"] > 0.13),

                "anger_pred": emo["anger"],
                "disgust_pred": emo["disgust"],
                "fear_pred": emo["fear"],
                "joy_pred": emo["joy"],
                "sadness_pred": emo["sadness"],
                "surprise_pred": emo["surprise"],
            }
        )

    df = pd.DataFrame(l)
    del l
    df.to_csv("pred_" + file)
    return df

In [ ]:
from os import listdir

def find_csv_filenames(path_to_dir, suffix=".csv"):
    filenames = listdir(path_to_dir)
    return [filename for filename in filenames if filename.endswith(suffix)]


filenames = find_csv_filenames("data")
done = find_csv_filenames("pred_data")
files = [l for l in filenames if l not in done]
for name in tqdm(files):
    if name not in done:
        f = "data/" + name
        print(f)
        try:
            predict_file(f)
        except:
            print("ERRO: " + f)

0it [00:00, ?it/s]
